# Qu'est-ce qu'il y a dans ce notebook ?

Fonctions permettant de calculer et créer un dictionnaire de similarité entre chaque occurence unique de mots  
Fonctions permettant d'extraire, pour chaque occurence unique, le mot le plus similaire et de récupérer son nombre d'occurence  
Fonctions permettant de recréer un dictionnaire d'occurence pour faciliter l'extraction en csv

In [1]:
debug_mode = False
serialized_data = True

In [3]:
import spacy
import json
#from ipynb.fs.defs.fonctions_preprocess import open_file
#from ipynb.fs.defs.fonctions_preprocess import serialisation_data

## Extraction et mise en forme du corpus

On extrait le corpus à partir du fichier souhaité (selon fichier en partie sérialisé ou non)  
Mise en format du fichier : nécessité de passer d'une liste de mot à une string pour que le module SpaCy fonctionne  
Sérialisation de l'objet string

In [4]:
def open_file(file):
    with open(file) as json_data:
        data = json.load(json_data)
    return data

In [5]:
if serialized_data == False:
    print("extracting whole data from noun corpus")
    data = open_file("nouns.json")
    print(len(data))
else:
    print("Extracting data_string from json file")
    corpus = open_file("corpus_string.json")
    print("corpus extracted")

Extracting data_string from json file
corpus extracted


In [6]:
if debug_mode:
    test = corpus[0:200]
    print(len(test))

#### Extraction des noms du tuple et création de la string

In [ ]:
def list_into_string(tuple_list):

    extract = []
    extract_s = ""

    #Permet d'enlever les doublons
    for tup in tuple_list:
        if tup[0] not in extract:
            extract.append(tup[0])
            
    print("Taille des occurences uniques : ", len(extract))
        
    for word in extract:
        extract_s += word
        extract_s += " "
        
    return extract_s

In [ ]:
if serialized_data == False:
    corpus = list_into_string(data)    

In [13]:
if serialized_data == False:
    serialisation_data (corpus, "corpus_string.json")

## Calcul de similarités via le module SpaCy

Chargement du module avec les vecteurs pré-entrainés `fr_core_news_md`  
Chargement du corpus dans deux documents pour s'assurer que tous les tokens seront lus ensemble  
Fonctions permettant le calcul de similarité et construction d'un dictionnaire de similarité  
Sérialisation des données

#### Chargement du module

In [7]:
nlp = spacy.load("fr_core_news_md")

In [8]:
nlp.max_length = 1500000

if debug_mode:
    print("debug mode : corpus test")
    doc1 = nlp(test)
    doc2 = nlp(test)
else:
    print("dealing with whole corpus")
    doc1 = nlp(corpus)
    doc2 = nlp(corpus)

dealing with whole corpus


#### Fonctions permettant le calcul de similarité

`calculate_similarity` calcule la similarité entre deux mots  
`construction_list_similarity` construit une liste de similarité constituée de tuples avec deux mots et leur similarité  
`extract_unique_referent` permet, à partir de la liste de similarité, de construire un dictionnaire avec, pour chaque occurence, ses similarités les plus fortes

In [9]:
def calculate_similarity (token1, token2):
    """Calculate Similarity between two tokens using SpaCy pre-trained model
    -> Two tokens
    <- similarity score between these two tokens"""

    similarity = token1.similarity(token2)
    
    if similarity >= 0.5 and similarity != 1 : return (token1, token2, similarity) 

In [10]:
def construction_liste_similarity(doc1, doc2):
    """Builds list of similarities. One element of the list is built so : (token1, token2, similarity)
    -> doc1 : list of words
    -> doc2 : other list of words
    <- list of tuples made of two words and their similarities
    """
    
    similarities = []
    
    for i in range (len(doc1)):
        if doc1[i].has_vector != False: tok = doc1[i]
        for k in range(len(doc2)):
            if doc2[k].has_vector != False : 
                similarity = calculate_similarity(tok, doc2[k])
                if similarity != None : similarities.append(similarity)
                    
    return similarities
    

In [11]:
def extract_unique_referents(doc1, doc2):
    """extract unique_referent using word_similarities
    -> string of words
    <- list of words and dictionnary of similarities
    doc_sim = {word1: [(word, similarity), ..., (word, similarity)], word2 : [(word, similarity), ... (word, similarity)], ... wordn : [(word, similarity), (word, similarity)]}
    
    We use word similarities using spacy module. We take similarities >= 0.5 to delete low similarities and "clean" corpus) 
    """
    
    humans_referent = []
    doc_sim = {}

    similarities = construction_liste_similarity(doc1, doc2)

    print("Similarities are done !")
    
    i=0
    for sim in similarities:
        i+=1
        if i%1000 == 0: print("Similarité n°", i, "/", len(similarities) )
        if sim[1] not in humans_referent: humans_referent.append((sim[1])) 
        if sim[0] not in doc_sim : 
            doc_sim[sim[0]] = []
        doc_sim[sim[0]].append((sim[1], sim[2]))
        

    print("Taille de humans_referent : ", len(humans_referent), " et taille de doc_sim : ", len(doc_sim))
    return (humans_referent, doc_sim)

In [12]:
extraction = extract_unique_referents(doc1, doc2)

KeyboardInterrupt: 

In [ ]:
if debug_mode:
    for dic in extraction[1]:
        print("dic :", dic, " -> ", extraction[1][dic])
        print()

#### Sérialisation des données

In [ ]:
def serialisation_data (data, title):
  """
  Serialize data in a json file
  -> Title mus be a string : title.json
  <- Save a file in desktop
  """

  with open(title, "w+") as file:
    json.dump(data, file)

In [ ]:
serialisation_data(extraction[0], "list_human_ref.json")

In [ ]:
serialisation_data(extraction[1], "dic_sim_human_ref.json")

## Extraction des référents humains

In [ ]:
def humans_occ (dict_occ, dict_sim):
    """Creates dict of occurences
    -> dict_occ : dictionnary of occcurences of each occurence
    -> dict_human_ref : dictionnary of similarities of each occurence
    <- dictionnary of occurences of most occurence
    """
    
    humans_ref = []
    humans_occ = {}
    
    
    for dic in dict_sim:
        sim_ord = sorted(dict_sim[dic])
        ref_max = sim_ord[0]
        if ref_max not in humans_ref : humans_ref.append(ref_max[1])
            
        if ref_max not in humans_occ: 
            humans_occ[ref_max] = dict_occ[ref_max]
        
    return dict_occ 